## Importar librerías iniciales e instancia de modelo de chat

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain,TransformChain
f = open('/home/iabd/clave_gpt.txt')
api_key = f.read().strip()
llm = ChatOpenAI(openai_api_key=api_key)

### Conectar a BD Vectores

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore

In [3]:
embedding_function = OpenAIEmbeddings(openai_api_key=api_key)

/tmp/ipykernel_5722/3525973800.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(openai_api_key=api_key)


In [4]:
vector_store_connection = SKLearnVectorStore(embedding=embedding_function, persist_path="../Bloque 3_Conectores de Datos/ejemplosk_embedding_db", serializer="parquet")

## Cargar cadena QA

In [5]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain #Opción que proporciona también la fuente de datos de la respuesta

In [6]:
chain = load_qa_chain(llm,chain_type='stuff') #chain_type='stuff' se usa cuando se desea una manera simple y directa de cargar y procesar el contenido completo sin dividirlo en fragmentos más pequeños. Es ideal para situaciones donde el volumen de datos no es demasiado grande y se puede manejar de manera eficiente por el modelo de lenguaje en una sola operación.

/tmp/ipykernel_5722/1964088940.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm,chain_type='stuff') #chain_type='stuff' se usa cuando se desea una manera simple y directa de cargar y procesar el contenido completo sin dividirlo en fragmentos más pequeños. Es ideal para situaciones donde el volumen de datos no es demasiado grande y se puede manejar de manera eficiente por el modelo de lenguaje en una sola operación.


In [7]:
question = "¿Qué pasó en el siglo de Oro?"

In [8]:
docs = vector_store_connection.similarity_search(question)

In [9]:
chain.run(input_documents=docs,question=question)

/tmp/ipykernel_5722/2437816174.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs,question=question)


'En el siglo de Oro español, que abarca desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681, hubo un florecimiento en las artes y las letras hispanas. Destacaron figuras como Miguel de Cervantes Saavedra, autor de Don Quijote de la Mancha, considerada la primera novela moderna. También surgieron géneros literarios como la picaresca con obras como el Lazarillo de Tormes. Grandes poetas, dramaturgos y prosistas como Quevedo, Góngora, Lope de Vega, Calderón de la Barca, entre otros, contribuyeron al esplendor cultural de la época.'

### Alternativa con método invoke

In [10]:
#Estructurar un diccionario con los parámetros de entrada
inputs = {
    "input_documents": docs,
    "question": question
}

In [11]:
chain.invoke(inputs)["output_text"]

'En el Siglo de Oro español, que abarca desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681, tuvieron lugar grandes avances en las artes y la literatura hispanas. Autores como Miguel de Cervantes Saavedra, que escribió "Don Quijote de la Mancha", y otros como Quevedo, Góngora, Lope, Calderón, y muchos más, crearon obras destacadas en la literatura, teatro y poesía. Fue un período donde florecieron las expresiones artísticas y el saber se acumuló y enseñó en prestigiosas universidades.'